In [ ]:
names_as_is = ['_-_past_year_likert_21', 
               '_-_behavior_choices_1',
               '_-_comments_21',
               '_-_demos',
               '_-_behavior_lk_21',
               '_-_behavior_lk_bl',
               '_-_generally_sem_bl',
               '_-_generally_sem_21',
               '_-_as_a_child',
               '_-_past_year_likert_bl']

## Sage Data Merge

TO DO:
    - Fix notes / documentation
    - Streamline the "as_is" dataframe, only code for df's in need of modification.   
    - Final Merge
    

NOTES FOR TEAM:
    - Following 'test_user' records are removed: ULoF3MM1nN gSWn9N D5bzYrfd8E ThpMV2Achc
    
Dataframe Columns Creation:
    
    bart_v4
        - Created columns for baseline, 21-day-assessment, bart250, bart0.25
    
    behavior_choices_4
        - Created columns for baseline (not reenrollment - has duplicates)
      
    delay_discounting
        - Created columns for baseline, 21-day-assessment, dd_time_6_month, dd_money_6_month, 
          dd_money_1_month, dd_time_1_year    
    
    discounting_raw
        - Created columns for baseline, 21-day-assessment, money, probability.
        
    evening_notification
        - Only included "baseline" data.  Did not include "reenrollment" or "set_evening_survey" because of
          duplicates.
        
    gonogo
        - NEED TO RECTIFY DUPLICATES (ksJM3Y)
        
    morning_notification
        - created columns for baseline only.
    
    PAM multiple
        - created columns for baseline and 21-day-assessment.
        
Missing Dataframes

        PAM_v2                        | ONLY INCLUDES AM_SURVEY AND PM_SURVEY
        EVENING SEM DIFF              | ONLY INCLUDE AM AND PM SURVEY
        Morning Behavior              | ONLY INCLUDES AM_SURVEY
        Morning Yesterday Sem Diff    | ONLY INCLUDES AM_SURVEY
        Morning Yesterday Likert      | ONLY INCLUDES AM_SURVEY
        Morning Semantics             | ONLY INCLUDES AM_SURVEY
        Comments_v2                   | Has duplicates
      
Unexplained duplicates (file, id's):
        - digital-marshmallow-as_a_child_likert_bl-v2_8.8.17.csv  | ThpMV2Achc
        - digital-marshmallow-behavior_choices_1_bl-v1_8.8.17.csv | ThpMV2Achc
        - digital-marshmallow-demographics-v2_8.8.17.csv          | ThpMV2Achc
        - digital-marshmallow-behavior_likert_bl-v2_8.8.17.csv    | ThpMV2Achc
        - digital-marshmallow-generally_sem_diff_bl-v2_8.8.17.csv | ThpMV2Achc
        - digital-marshmallow-past_year_likert_bl-v3_8.8.17.csv   | ThpMV2Achc
        - digital-marshmallow-comments-v2_8.8.17.csv              | CdtW9X Db8hWK

In [1]:
import os, pandas as pd, numpy as np, re
from functools import reduce
pd.set_option('display.max_columns', 100)

#NOTE TO SELF: redo the relative paths.
sage = '/home/wraikes/Dropbox/partnership/DMTBilly data - Copy/Sage Data'
os.chdir(sage)

In [4]:
def remove_dupes(df):
    #'Must confirm removal of these dupes, especially ThpMV2Achc
    test_users = ['ULoF3MM1nN', 'gSWn9N', 'D5bzYrfd8E', 'ThpMV2Achc']
    return df[~df.externalId.isin(test_users)]

def dupe_check(df):
    return len(df.externalId) == len(df.externalId.unique()) and len(df.externalId) > 0

def new_cols(df, append):
    # Remove: 'metadata.json.' and 'data.json.'.
    col_re = re.compile('metadata.json.|data.json.')
    df.columns = list(map(lambda x: re.sub(col_re, '', x), df.columns))
    
    # Append df identifier to columns.
    df.columns = list(map(lambda x: x + append, df.columns))
    df = df.rename(columns = {'externalId' + append: 'externalId'})

    return df

def create_new_df(df, func, attributes):
    dfs = []
    
    for att in attributes:
        if len(att) == 3:
            new_df = func(df, att[1], att[2])
        else:
            new_df = func(df, att[1])
        new_df = new_cols(new_df, att[0])
        dfs.append(new_df)

    df_merge = reduce(lambda left, right: pd.merge(left, right, how = 'outer',
                                                   on='externalId'), 
                                                   dfs)
    
    return df_merge

In [5]:
test_users = ['ULoF3MM1nN', 'gSWn9N', 'D5bzYrfd8E', 'ThpMV2Achc']
files_to_exclude = ['digital-marshmallow-status_8.8.17.csv',
                    'digital-marshmallow-appVersion_8.8.17.csv']

files_as_is = []

for file in os.listdir():
    if file not in files_to_exclude:
        df = pd.read_csv(file)
        df = remove_dupes(df)
        if dupe_check(df):
            files_as_is.append(file)
        else:
            continue

In [6]:
names_as_is = ['_-_past_year_likert_21', 
               '_-_behavior_choices_1',
               '_-_comments_21',
               '_-_demos',
               '_-_behavior_lk_21',
               '_-_behavior_lk_bl',
               '_-_generally_sem_bl',
               '_-_generally_sem_21',
               '_-_as_a_child',
               '_-_past_year_likert_bl']

In [7]:
new_dfs = {}

for file, name in zip(files_as_is, names_as_is):
    df = pd.read_csv(file)
    df = remove_dupes(df)
    df = new_cols(df, name)
    new_dfs[name] = df

In [8]:
all([dupe_check(df) for name, df in new_dfs.items()])

True

### DataFrame: Bart_V4

In [9]:
bart_v4 = pd.read_csv('digital-marshmallow-bart-v4_8.8.17.csv')
bart_v4 = remove_dupes(bart_v4)

dupe_check(bart_v4)

False

In [10]:
def bart_dataframe(df, var, survey):
    cols = df.columns
    new_df = pd.DataFrame(columns = cols)
    
    for ix, row in df.iterrows():
        if row['data.json.variable_label'] == var and row['metadata.json.taskIdentifier'] == survey:
            new_df = new_df.append(row, ignore_index=True)
    
    return new_df

In [11]:
bart_attributes = [
    ['_-_bart_v4_bl_0.25', 'BART0.25', 'baseline'],
    ['_-_bart_v4_bl_250', 'BART250.00', 'baseline'],
    ['_-_bart_v4_21_0.25', 'BART250.00', '21-day-assessment'],
    ['_-_bart_v4_21_250', 'BART250.00', '21-day-assessment']
]

In [12]:
bart_v4 = create_new_df(bart_v4, bart_dataframe, bart_attributes)

In [13]:
dupe_check(bart_v4)

True

### DataFrame: Behavior_choices_4

In [14]:
behavior_4 = pd.read_csv('digital-marshmallow-behavior_choices_4_bl-v2_8.8.17.csv')
behavior_4 = remove_dupes(behavior_4)

dupe_check(behavior_4)

False

In [15]:
def behave_dataframe(df, survey):
    cols = df.columns
    new_df = pd.DataFrame(columns = cols)
    
    for ix, row in df.iterrows():
        if row['metadata.json.taskIdentifier'] == survey:
            new_df = new_df.append(row, ignore_index=True)
    
    return new_df

In [16]:
behavior_4_attributes = [
    ['_-_behavior_4_bl', 'baseline'],
]

In [17]:
behavior_4 = create_new_df(behavior_4, behave_dataframe, behavior_4_attributes)

In [18]:
dupe_check(behavior_4)

True

### DataFrame: Delay Discounting

In [26]:
delay = pd.read_csv('digital-marshmallow-delay_discounting_raw-v6_8.8.17.csv')
delay = remove_dupes(delay)

dupe_check(delay)

False

In [27]:
def delay_dataframe(df, var, survey):
    cols = df.columns
    new_df = pd.DataFrame(columns = cols)
    
    for ix, row in df.iterrows():
        if row['data.json.variableLabel'] == var and row['metadata.json.taskIdentifier'] == survey:
            new_df = new_df.append(row, ignore_index=True)
    
    return new_df

In [28]:
bl = 'baseline'
_21 = '21-day-assessment'

delay_attributes = [
    ['_-_delay_bl_time_6_month', 'dd_time_6_month', bl],
    ['_-_delay_bl_money_6_month', 'dd_money_6_month', bl],
    ['_-_delay_bl_money_1_month', 'dd_money_1_month', bl],
    ['_-_delay_bl_time_1_year', 'dd_time_1_year', bl],
    ['_-_delay_21_time_6_month', 'dd_time_6_month', _21],
    ['_-_delay_21_money_6_month', 'dd_money_6_month', _21],
    ['_-_delay_21_money_1_month', 'dd_money_1_month', _21],
    ['_-_delay_21_time_1_year', 'dd_time_1_year', _21]
]

delay = create_new_df(df=delay, func=delay_dataframe, attributes=delay_attributes)

In [29]:
dupe_check(delay)

True

### DataFrame: Discounting Raw

In [37]:
discount = pd.read_csv('digital-marshmallow-discounting_raw-v2_8.8.17.csv')
discount = remove_dupes(discount)

dupe_check(discount)

False

In [38]:
def discount_dataframe(df, var, survey):
    cols = df.columns
    new_df = pd.DataFrame(columns = cols)
    
    for ix, row in df.iterrows():
        if row['data.json.variableLabel'] == var and row['metadata.json.taskIdentifier'] == survey:
            new_df = new_df.append(row, ignore_index=True)
    
    return new_df

In [39]:
bl = 'baseline'
_21 = '21-day-assessment'

discount_attributes = [
    ['_-_discount_bl_money', 'pd_constant_money', bl],
    ['_-_discount_bl_prob', 'pd_constant_probabiliy', bl],
    ['_-_discount_21_money', 'pd_constant_money', _21],
    ['_-_discount_21_prob', 'pd_constant_probability', _21]
]

discount = create_new_df(df=discount, func=discount_dataframe, attributes=discount_attributes)

In [40]:
dupe_check(discount)

True

### DataFrame: Evening Notification

In [41]:
evening_note = pd.read_csv('digital-marshmallow-evening_notification_time-v2_8.8.17.csv')
evening_note = remove_dupes(evening_note)

dupe_check(evening_note)

False

In [42]:
def evening_note_dataframe(df, survey):
    cols = df.columns
    new_df = pd.DataFrame(columns = cols)
    
    for ix, row in df.iterrows():
        if row['metadata.json.taskIdentifier'] == survey:
            new_df = new_df.append(row, ignore_index=True)
    
    return new_df

In [43]:
evening_note_attributes = [
    ['_-_evening_note_bl', 'baseline']
]

evening_note_bl = create_new_df(evening_note, evening_note_dataframe, evening_note_attributes)

In [44]:
dupe_check(evening_note_bl)

True

### DataFrame: GoNoGo - PLACEHOLDER (Extra Record - ksJM3Y)

In [45]:
gonogo = pd.read_csv('digital-marshmallow-goNoGo-v2_8.8.17.csv')
gonogo = remove_dupes(gonogo)

dupe_check(gonogo)

False

In [46]:
def gonogo_dataframe(df, var, survey):
    cols = df.columns
    new_df = pd.DataFrame(columns = cols)
    
    for ix, row in df.iterrows():
        if row['data.json.variable_label'] == var and row['metadata.json.taskIdentifier'] == survey:
            new_df = new_df.append(row, ignore_index=True)
    
    return new_df

In [47]:
bl = 'baseline'
_21 = '21-day-assessment'

gonogo_attributes = [
    ['_-_gonogo_bl_stable', 'go_no_go_stable_stimulus_active_task', bl],
    ['_-_gonogo_21_variable', 'go_no_go_variable_stimulus_active_task', bl],
    ['_-_gonogo_bl_stable', 'go_no_go_stable_stimulus_active_task', _21],
    ['_-_gonogo_21_variable', 'go_no_go_variable_stimulus_active_task', _21]
]

gonogo = create_new_df(df=gonogo, func=gonogo_dataframe, attributes=gonogo_attributes)

In [48]:
dupe_check(gonogo)

False

In [49]:
gonogo.externalId[gonogo.externalId.duplicated()]

12    ksJM3Y
Name: externalId, dtype: object

### DataFrame: Morning Notifications

In [55]:
morning_note = pd.read_csv('digital-marshmallow-morning_notification_time-v3_8.8.17.csv')
morning_note = remove_dupes(morning_note)

dupe_check(morning_note)

False

In [56]:
def morning_note_dataframe(df, survey):
    cols = df.columns
    new_df = pd.DataFrame(columns = cols)
    
    for ix, row in df.iterrows():
        if row['metadata.json.taskIdentifier'] == survey:
            new_df = new_df.append(row, ignore_index=True)
    
    return new_df

In [57]:
morning_note_attributes = [
    ['_-_morning_note_bl', 'baseline']
]

morning_note = create_new_df(morning_note, morning_note_dataframe, morning_note_attributes)

In [58]:
dupe_check(morning_note)

True

### DataFrame: PAM Multiple

In [59]:
pam_mult = pd.read_csv('digital-marshmallow-pam_multiple-v2_8.8.17.csv')
pam_mult = remove_dupes(pam_mult)

dupe_check(pam_mult)

False

In [60]:
def pam_mult_dataframe(df, survey):
    cols = df.columns
    new_df = pd.DataFrame(columns = cols)
    
    for ix, row in df.iterrows():
        if row['metadata.json.taskIdentifier'] == survey:
            new_df = new_df.append(row, ignore_index=True)
    
    return new_df

In [61]:
pam_mult_attributes = [
    ['_-_pam_mult_bl', 'baseline'],
    ['_-_pam_mult_21', '21-day-assessment']
]

pam_mult = create_new_df(pam_mult, pam_mult_dataframe, pam_mult_attributes)

In [62]:
dupe_check(pam_mult)

True

### Final Merge of All Sage Data

In [64]:
new_dfs['pam'] = pam_mult
new_dfs['morning_note'] = morning_note
new_dfs['evening_note_bl'] = evening_note_bl
new_dfs['discount'] = discount
new_dfs['delay'] = delay
new_dfs['bart'] = bart_v4
new_dfs['behave'] = behavior_4

In [71]:
final_df = reduce(lambda left, right: pd.merge(left, right, how = 'outer',
                                               on='externalId'), 
                  new_dfs.values())

In [74]:
dupe_check(final_df)

True

In [76]:
os.chdir('/home/wraikes/Programming/Partnership/dmt/merged_data/')
final_df.to_csv('FINAL_SAGE.csv')